In [41]:
import h5py
import pickle
import numpy as np
import pandas as pd
import glob
import os
import datetime

## 2021-11-21 Writing FAS info to h5py file

In [29]:
data_path = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate'
FAS_peak_detections_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_detections.obj'
FAS_plot_data_padded_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_plot_data_padded.obj'
FAS_plot_data_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_plot_data.obj'
FAS_prominences_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_prominences.obj'
FAS_ranges_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_ranges.obj'
FAS_selected_date_ranges_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_selected_date_ranges.obj'

FAS_h5py_file = os.path.join(data_path, 'FAS_peak_analysis.hdf5')


In [9]:
with open(FAS_peak_detections_file, 'rb') as f:
    FAS_peak_detections = pickle.load(f)

In [18]:
with h5py.File(FAS_h5py_file, 'a') as f:
    g = f.create_group('peak_detections')
    for entry in FAS_peak_detections:
        h = g.create_group(entry[0])
        d = h.create_dataset('peak_locations', data=entry[1])
        dd = h.create_dataset('peak_heights', data=entry[2]['peak_heights'])
        ddd = h.create_dataset('prominences', data=entry[2]['prominences'])
        dddd = h.create_dataset('left_bases', data=entry[2]['left_bases'])
        ddddd = h.create_dataset('right_bases', data=entry[2]['right_bases'])

In [20]:
with open(FAS_plot_data_file, 'rb') as f:
    FAS_plot_data = pickle.load(f)

In [22]:
with open(FAS_plot_data_padded_file, 'rb') as f:
    FAS_plot_data_padded = pickle.load(f)

In [25]:
FAS_plot_data_padded.to_hdf(FAS_h5py_file, 'plot_data')

In [27]:
with open(FAS_prominences_file, 'rb') as f:
    FAS_prominences = pickle.load(f)

In [28]:
FAS_prominences

[459.18518518518516,
 2637.779487179487,
 48.57142857142857,
 271.76,
 1692.6190476190477,
 293.9761904761905,
 295.73809523809524,
 2425.0714285714284,
 793.5952380952381,
 670.0714285714286]

In [30]:
with open(FAS_selected_date_ranges_file, 'rb') as f:
    FAS_selected_date_ranges = pickle.load(f)

In [32]:
FAS_selected_date_ranges

[(datetime.datetime(2017, 11, 15, 0, 0), datetime.datetime(2018, 5, 29, 0, 0)),
 (datetime.datetime(2018, 11, 21, 0, 0), datetime.datetime(2019, 1, 2, 0, 0)),
 (datetime.datetime(2020, 2, 3, 0, 0), datetime.datetime(2020, 3, 16, 0, 0))]

In [43]:
FAS_selected_date_ranges = [
    (datetime.datetime.strftime(i[0], '%Y-%m-%d'),datetime.datetime.strftime(i[1], '%Y-%m-%d')) for i in FAS_selected_date_ranges
]

In [44]:
FAS_selected_date_ranges

[('2017-11-15', '2018-05-29'),
 ('2018-11-21', '2019-01-02'),
 ('2020-02-03', '2020-03-16')]

In [31]:
with open(FAS_ranges_file, 'rb') as f:
    FAS_ranges = pickle.load(f)

In [38]:
print(FAS_ranges[1])
print(FAS_ranges[4])
print(FAS_ranges[7])

(datetime.datetime(2017, 11, 15, 0, 0), datetime.datetime(2018, 5, 29, 0, 0))
(datetime.datetime(2018, 11, 21, 0, 0), datetime.datetime(2019, 1, 2, 0, 0))
(datetime.datetime(2020, 2, 3, 0, 0), datetime.datetime(2020, 3, 16, 0, 0))


In [45]:
FAS_ranges = [
    (datetime.datetime.strftime(i[0], '%Y-%m-%d'),datetime.datetime.strftime(i[1], '%Y-%m-%d')) for i in FAS_ranges
]

In [46]:
dt = h5py.special_dtype(vlen=str) 
FAS_ranges = np.array(FAS_ranges, dtype=dt)
FAS_selected_date_ranges = np.array(FAS_selected_date_ranges, dtype=dt)

In [53]:
with h5py.File(FAS_h5py_file, 'a') as f:
    g = f.require_group('segments')
    d = g.create_dataset('prominences', data=FAS_prominences)
    dd = g.create_dataset('ranges', data=FAS_ranges)
    dd.attrs.update({'selected_indices':'1,4,7'})
    ddd = g.create_dataset('selected_ranges', data=FAS_selected_date_ranges)
    metadata = {
        'description': 'Segments of the full archive search activity, grouped by detected peaks'
    }
    g.attrs.update(metadata)

In [54]:
with h5py.File(FAS_h5py_file, 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['peak_detections', 'plot_data', 'segments']>


In [55]:
def allkeys(obj):
    "Recursively find all keys in an h5py.Group."
    keys = (obj.name,)
    if isinstance(obj, h5py.Group):
        for key, value in obj.items():
            if isinstance(value, h5py.Group):
                keys = keys + allkeys(value)
            else:
                keys = keys + (value.name,)
    return keys

In [58]:
def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

In [62]:
with h5py.File(FAS_h5py_file, 'r') as f:
    x = get_dataset_keys(f)

In [63]:
import pprint

In [64]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(x)

[   'peak_detections/anakaman/left_bases',
    'peak_detections/anakaman/peak_heights',
    'peak_detections/anakaman/peak_locations',
    'peak_detections/anakaman/prominences',
    'peak_detections/anakaman/right_bases',
    'peak_detections/balancetonporc/left_bases',
    'peak_detections/balancetonporc/peak_heights',
    'peak_detections/balancetonporc/peak_locations',
    'peak_detections/balancetonporc/prominences',
    'peak_detections/balancetonporc/right_bases',
    'peak_detections/cuentalo/left_bases',
    'peak_detections/cuentalo/peak_heights',
    'peak_detections/cuentalo/peak_locations',
    'peak_detections/cuentalo/prominences',
    'peak_detections/cuentalo/right_bases',
    'peak_detections/enazeda/left_bases',
    'peak_detections/enazeda/peak_heights',
    'peak_detections/enazeda/peak_locations',
    'peak_detections/enazeda/prominences',
    'peak_detections/enazeda/right_bases',
    'peak_detections/gamani/left_bases',
    'peak_detections/gamani/peak_heights',

## 2021-11-21 Writing timeline completeness material to h5py file

In [71]:
timeline_completeness_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/timeline_completeness.obj'

In [72]:
with open(timeline_completeness_file, 'rb') as f:
    timeline_completeness = pickle.load(f)

In [74]:
import re
def converter(input_tuple):
    x, y = input_tuple
    x = re.split('[_.]',x)[-2]
    y = datetime.datetime.isoformat(y)
    return x,y

In [76]:
with h5py.File(f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/timeline_completeness.hdf5','w') as f:
    g = f.create_group('group_1')
    for pair in timeline_completeness:
        pair = converter(pair)
        g.create_dataset(pair[0], data=pair[1])